# Principal Components Analysis: PCA

Extract from my PhD thesis: http://dspace.ut.ee/bitstream/handle/10062/82088/molinaro_ludovica.pdf?sequence=6&isAllowed=y

When it comes to analysing the vast amount of information of the genetic data, dimensionality reduction techniques are essential tools. They comprise a set of approaches that aim to understand the relationship between multiple attributes (for example SNPs) of an entity (such as individuals) and assess their relevance in describing the set while minimising the information loss (Jolliffe and Cadima 2016).   
Among the dimensionality reduction techniques, Principal Component Analysis (PCA) is one of the most used in population genetics since Menozzi et al pioneered its usage to study genetic variation (Novembre and Stephens 2008; Menozzi P., Piazza A., and Cavalli-Sforza L. 1978). PCA does not operate on a priori information: it is an optimal exploratory tool to make predictive models.  


#### The base assumption is that 
genotypes will cluster together in the PC space according to their similarity, so individuals with a recent shared ancestry should fall closer together than more distantly related individuals (Schraiber and Akey 2015). As demonstrated by McVean 2009 the location of samples on the PC space can be related to the mean time of coalescence between pairs of samples. 

#### On the other hand, 
if multiple demographic models have the same effect on mean coalescence times, it is difficult to define what kind of event characterised the population under study (McVean 2009; Novembre and Stephens 2008). It follows that despite allowing for a wide range of inferences, interpreting past demographic events from PCA patterns is challenging and should be done with caution. 

#### Biases
PCA projections depend strongly on the sample size, as sample size differences between populations will distort the projection space, but also sampling location and ascertainment of samples may cause biases. 

#### Projection 
An additional useful feature of PCA is to define the PC space with selected samples and then project the samples of interest. Projecting samples on an already defined PC space avoids potentially skewing the analyses when the sample size of the target group is significantly different from the other set, or when the target samples are characterised by a substantial missing data, as commonly happens with ancient data. Additionally, projecting samples is useful when they are thought to be admixed, in fact by projecting the admixed individuals in the reference populations defined PC space, it is also possible to identify the admixture proportions (McVean 2009). 

## Preparing the dataset

To run PCA we first need to remove SNPs in Linkage Disequilibrium (LD).  
Using the software plink v1.9, we are going to first indentifying the SNPs that:  
- show a pariwise r^2 > `0.1`
- in a genomic window of `50` SNPs,  
- shifted by `10` SNPs at the end of each step  

All the SNPs that exceed the 0.1 threshold, will then be removed

In [ ]:
! plink --bfile dataset/1KGs_chr1_maf \
        --indep-pairwise 50 10 0.1 \
        --out dataset/SNPs_inLD

The command `--indep-pairwise` will create two files: X.prune.in and X.prune.out, where as X plink will set the name we indicated as an argument of `--output`

We will now remove the SNPs that pass the 0.1 threshold, with `--exclude` and the file.prune.out created by the `--indep-pairwise` command. 

In [ ]:
! plink --bfile dataset/1KGs_chr1_maf \
        --exclude dataset/SNPs_inLD.prune.out \
        --make-bed \
        --out dataset/1KGs_chr1_maf_pruned

### Preparing the files
#### Converting plink files to EIGENSOFT format

We are going to run the PCA with the EIGENSOFT software called smartpca. First, we have to convert the plink files in EIGENSTRAT format, so that the program smartpca will know how to read them.  
To convert the files we are going to use a software available in EIGENSTRAT, called **convertf**. The basic syntax of **convertf** is:

`convertf -p file.par ` 

The file.par contains the information needed to properly convert your file.  In the directory .scripts/ you will find a bash script that will automatically create a convertf parfile, given an input file name and an output file name. The first three lines of the par file list the 3 binary plink files, the fourth row indicate the final output format (EIGENSTRAT). The geno/snp/ind file are the EIGENSTRAT files that correspont to bed/bim/fam files.  

The bash command can be ran this way:

In [7]:
! bash scripts/BED2EIG.sh dataset/1KGs_chr1_maf_pruned 1KGs_chr1_maf_pruned_converted

In [8]:
! sed -i 's/-9/1/g' dataset/1KGs_chr1_maf_pruned.fam

With the par file available, we are now ready to run convertf

In [7]:
! convertf -p convertf_1KGs_chr1_maf_pruned_converted.par

parameter file: convertf_1KGs_chr1_maf_pruned_converted.par
genotypename: dataset/1KGs_chr1_maf_pruned.bed
snpname: dataset/1KGs_chr1_maf_pruned.bim
indivname: dataset/1KGs_chr1_maf_pruned.fam
outputformat: EIGENSTRAT
genotypeoutname: 1KGs_chr1_maf_pruned_converted.geno
snpoutname: 1KGs_chr1_maf_pruned_converted.snp
indivoutname: 1KGs_chr1_maf_pruned_converted.ind
## convertf version: 5722
genetic distance set from physical distance
genotype file processed
before compress: snps: 31437 indivs: 3202
after compress: snps: 29301 indivs: 3202
numvalidind:   3202  maxmiss: 3202001
eigenstrat output
numsnps output: 29301
##end of convertf:       45.075 seconds cpu       19.599 Mbytes in use


## Running smartpca without projection

Now that we have the EIGENSOFT-format files, we can run thw software **smartpca** to run the principal components analysis. To do so, we must first create a par file for the software **smartpca** as well. An automated script to create a smartpca par file is available in scripts/SMARTPCA.sh, and can be used as follows:

In [8]:
! bash scripts/SMARTPCA.sh 1KGs_chr1_maf_pruned_converted

In [9]:
! more smartpca_1KGs_chr1_maf_pruned_converted.par

genotypename:    1KGs_chr1_maf_pruned_converted.geno
snpname:         1KGs_chr1_maf_pruned_converted.snp
indivname:       1KGs_chr1_maf_pruned_converted.ind
evecoutname:	1KGs_chr1_maf_pruned_converted.pca.evec
evaloutname:	1KGs_chr1_maf_pruned_converted.eval
altnormstyle: NO
numoutevec: 10
numoutlieriter: 5
numoutlierevec: 10
lsqproject: YES
poplistname: pop_list.txt


Since we are running PCA without projection, we can remove the last two lines of the par file

In [10]:
! head -n -2 smartpca_1KGs_chr1_maf_pruned_converted.par > tmp.par
! mv tmp.par smartpca_1KGs_chr1_maf_pruned_converted.par
! more smartpca_1KGs_chr1_maf_pruned_converted.par

genotypename:    1KGs_chr1_maf_pruned_converted.geno
snpname:         1KGs_chr1_maf_pruned_converted.snp
indivname:       1KGs_chr1_maf_pruned_converted.ind
evecoutname:	1KGs_chr1_maf_pruned_converted.pca.evec
evaloutname:	1KGs_chr1_maf_pruned_converted.eval
altnormstyle: NO
numoutevec: 10
numoutlieriter: 5
numoutlierevec: 10


#### We are now ready to run smartpca on our dataset.

In [ ]:
! smartpca -p smartpca_1KGs_chr1_maf_pruned_converted.par

#### Inside the output files

In [12]:
! head 1KGs_chr1_maf_pruned_converted.pca.evec

           #eigvals:   254.240    98.886    31.465    23.809     5.018     4.818     4.307     4.211     3.943     3.830 
         GBR:HG00096    -0.0116     -0.0219      0.0046     -0.0162      0.0013     -0.0125     -0.0136      0.0194      0.0059      0.0080          Control
         GBR:HG00097    -0.0100     -0.0229      0.0034     -0.0182     -0.0049      0.0016      0.0157      0.0219     -0.0401     -0.0221          Control
         GBR:HG00099    -0.0112     -0.0243      0.0048     -0.0187      0.0058     -0.0069     -0.0224      0.0219      0.0106     -0.0135          Control
         GBR:HG00100    -0.0108     -0.0234      0.0046     -0.0150     -0.0054     -0.0061      0.0148      0.0122     -0.0246      0.0024          Control
         GBR:HG00101    -0.0119     -0.0239      0.0080     -0.0162     -0.0152     -0.0280     -0.0059      0.0389      0.0120      0.0118          Control
         GBR:HG00102    -0.0103     -0.0242      0.0065     -0.0206      0.0112      0.0160  

The `.evec` file is a dataframe with N columns and M rows.  
The number of columns correspond to the samples ID, followed by the N PC we requested in numoucevec (see parfile), and finally one last column, that in the non-projected run is not useful.
While the number or rows, correspond to the number of samples on which the PCA was performed. The `.evec` file is the file with the eigenvectors (the PCA) that we will plot

The `.eval` file is a one column file, containing the eigenvalues. For this tutorial, we won't look into the .eval file

### Basic plot

## Running smartpca with projection

To run PCA with projection, let's first create the smartpca par file. The two key parameters for a projected run are: A) lsqproject:YES, B) poplistname: file.  
An automated script to create a smartpca par file is available in scripts/SMARTPCA.sh, and can be used as follows:

In [13]:
! bash scripts/SMARTPCA.sh 1KGs_chr1_maf_pruned_converted

We can use the .ind file we prepared for the not-projected run. However,  the .ind file should be modified so that the software can select the groups to build the PC space on ('Controls') and the groups to project ('Cases'). Let's say that we want GBR, ESN and CHS as `Controls`, and all other groups will be projected on them (therefore `Cases`). 

In [16]:
! sed -r 's/:/\t/g' 1KGs_chr1_maf_pruned_converted.ind > tmp.ind
! awk '{ if ($1 == "GBR" || $1 == "ESN" || $1 == "CHS") print $1,$2,$3,$4="Control"; else print  $1,$2,$3,$4="Case" }' tmp.ind > tmp2.ind
! awk '{print $1":"$2,$3,$4}' tmp2.ind > 1KGs_chr1_maf_pruned_converted.ind
! rm tmp*

Now we should create the pop_list.txt file, containing the list of populations that should be used to build the pc space. Since we clustered them together as `Control`, the only thing we have to do now is to create a pop_list.txt file with the word `Control` inside.

In [18]:
! echo "Control" > pop_list.txt

We are now ready to run smartpca!

In [ ]:
! bash scripts/SMARTPCA.sh 1KGs_chr1_maf_pruned_converted 
! smartpca -p smartpca_1KGs_chr1_maf_pruned_converted.par

#### Inside the output files

In [20]:
! head 1KGs_chr1_maf_pruned_converted.pca.evec

           #eigvals:    44.174    17.944     1.897     1.894     1.880     1.859     1.850     1.833     1.811     1.791 
         GBR:HG00096    -0.0250     -0.0870     -0.0048      0.0086     -0.0070     -0.0048     -0.0089     -0.0014     -0.0020     -0.0359          Control
         GBR:HG00097    -0.0212     -0.0866      0.0071     -0.0321      0.0170     -0.0012      0.0031      0.0044      0.0105     -0.0089          Control
         GBR:HG00099    -0.0235     -0.0933     -0.0146      0.0144      0.0109     -0.0116     -0.0025     -0.0173      0.0094      0.0232          Control
         GBR:HG00100    -0.0225     -0.0896     -0.0051     -0.0103     -0.0101     -0.0094     -0.0139     -0.0078     -0.0084      0.0131          Control
         GBR:HG00101    -0.0248     -0.0932     -0.0420     -0.0095      0.0039      0.0159     -0.0079     -0.0123     -0.0244     -0.0043          Control
         GBR:HG00102    -0.0212     -0.0920      0.0034      0.0095      0.0020     -0.0097  

The `.evec` file is a dataframe with N columns and M rows.  
The number of columns correspond to the samples ID, followed by the N PC we requested in numoucevec (see parfile), and finally one last column, that in the non-projected run is not useful.
While the number or rows, correspond to the number of samples on which the PCA was performed. 

#### The `.evec` file is the file with the eigenvectors (the PCA) that we will plot

The `.eval` file is a one column file, containing the eigenvalues. For this tutorial, we won't look into the .eval file.

### Basic plot